In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install spacy
!pip install scispacy
!python -m spacy download en
!pip install num2words
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Using cached spacy-3.0.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.3 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.3
    Uninstalling spacy-3.1.3:
      Successfully uninstalled spacy-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.1.0 requires spacy<3.2.0,>=3.1.0, but you have spacy 3.0.7 which is incompatible.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7 MB 74 kB/s             
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.1.0
    Uninstalling en-core-web-sm-3.1.0:
      Successfully uninstalled

In [17]:
from scispacy.abbreviation import AbbreviationDetector
from spacy.symbols import ORTH,NORM
from collections import defaultdict
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from num2words import num2words
from numpy import linalg as la
from array import array
import numpy as np
import collections
import string
import spacy
import time
import math
import json
import nltk
import ast
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
spacy.load('en_core_web_sm')

In [19]:
#Function to assess wheter a string is a number or not 
def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [20]:
docs_path = '/content/drive/Shareddrives/Information Retriveal and Web Analysis/P1/inputs/dataset_tweets_WHO.txt'
#docs_path = 'inputs/dataset_tweets_WHO.txt'

dictionary = []
with open(docs_path) as fp:
    tweets = json.loads(fp.read())
    
print("Total number of tweets in the database: {}".format(len(tweets)))

Total number of tweets in the database: 2399


In [21]:
def build_terms(tweet):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    line -- string (text) to be preprocessed
    
    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    print("\n*****Original Tweet*****")
    print(tweet)

    #Removing links
    tweet=  tweet.split()
    tweet=' '.join([word for word in tweet if "http" not in word])
    
    #removing emojis
    tweet = remove_emojis(tweet)

    print("\n*****Without Emojis and Links*****")
    print(tweet)

    #Detecting hashatags
    tweet = re.sub(r"(\s|^)\#(\S)", r"\1 hashtag \2", tweet)
    tweet = re.sub(r"(\s|^)\@(\S)", r"\1 atsign \2", tweet)

    punctuation_marks = set(string.punctuation+"…"+"’"+"“"+"”")-set([".","-","/","%","$","\n"])
    tweet = ''.join(ch for ch in tweet if ch not in punctuation_marks)
    
    #we just replace any slash to a white space.
    tweet = re.sub("/", " ", tweet)

    #We replace any point placed at the end of a number into a white space (if we didn't do so, we would have problems in the "num2words" step)
    tweet = re.sub(r"(\d)\.(\D)", r"\1 \2",tweet)

    print("\n*****Without Hashtag and Slash*****")
    print(tweet)
    
    #Replacing any quantity with the dollar sign to the quantity followed by the word "dollars".
    tweet = re.sub(r'\$([0-9]+\.?[0-9]*)', '\g<1> dollars', tweet)
    
    #Replacing any "%" sign to the word "percent"
    tweet = re.sub("%"," percent", tweet)

    #Replacing any "-" sign to a white space
    tweet = re.sub(r"[-–]", " ", tweet)
    
    #Any quantity followed by "s" is now followed by the "seconds" word (instead of the "s" char)
    tweet = re.sub(r"(\d)s",r"\1 seconds",tweet)  

    #"Num2words" step. We iterate through every word of the current text and we check if the word is actually a number (by using
    #the function defined above). If it is a number, we replace it by its word representation. If not, we just leave it as it is.
    words = tweet.split(" ")
    tweet = ""
    for word in words:
        if is_number(word):
            try:
                word = num2words(float(word))
            except:
                print("Error trying to transform " + str(word) + " into number")
        tweet+=word+" " 

    #After converting numbers to words, commas and hyphens can appear again, so we have to remove them with the same code used before    
    tweet = re.sub(r"[-–]", " ", tweet)
    tweet = re.sub(",", "", tweet) 

    #After converting numbers to words, we can also remove all the remaining dots
    tweet = ''.join(ch for ch in tweet if ch !=".")

    #If we had more than one consecutive white space, we replace it by a single one
    tweet = re.sub(' +', ' ',tweet)

    #For every acronym detected, we replace any occurence in the text by its expansion:
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("abbreviation_detector")

    #WHO special case
    special_case = [{ORTH: "W"}, {ORTH: "H"},{ORTH:"O"}]
    nlp.tokenizer.add_special_case("WHO", special_case)

    #Eliminating acronyms
    doc = nlp(tweet)
    altered_tok = [tok.text for tok in doc]
    for abrv in doc._.abbreviations:
        altered_tok[abrv.start] = str(abrv._.long_form)

    tweet = " ".join(altered_tok)
    
    #Transform WHO to World Health Organization
    tweet=  tweet.split()
    tweet= ["WORLD" if word == "W" else word for word in tweet]
    tweet= ["HEALTH" if word == "H" else word for word in tweet]
    tweet= " ".join(["ORGANIZATION" if word == "O" else word for word in tweet])

    #converting all the characters to lowercase
    tweet = tweet.lower()
    
    #removing stop words
    tweet = tweet.split()
    stop_words = set(stopwords.words("english"))
    tweet=[word for word in tweet if word not in stop_words]  ##eliminate the stopwords 

    print("\n*****Removing Stop words*****")
    print(tweet)
    
    #stemming
    stemmer = PorterStemmer()
    tweet=[stemmer.stem(word) for word in tweet] ## perform stemming 
    
    print("\n*****Stemming->Final Result*****")
    print(tweet)

    ## END CODE
    return tweet

In [22]:
for tweet, tweet_info in tweets.items():
    #print(tweet_info["full_text"])
    build_terms(tweet_info["full_text"])
    print("")
    time.sleep(1)


*****Original Tweet*****
It's International Day for Disaster Risk Reduction

#OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response.

Start learning today &amp; be #Ready4Response:
👉 https://t.co/hBFFOF0xKL https://t.co/fgZY22RWuS

*****Without Emojis and Links*****
It's International Day for Disaster Risk Reduction #OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response. Start learning today &amp; be #Ready4Response: 

*****Without Hashtag and Slash*****
Its International Day for Disaster Risk Reduction  hashtag OpenWHO has launched a multi-tiered core curriculum to help equip you with the competencies needed to work within public health emergency response. Start learning today amp be  hashtag Ready4Response 


/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:216: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)



*****Removing Stop words*****
['international', 'day', 'disaster', 'risk', 'reduction', 'hashtag', 'openwho', 'launched', 'multi', 'tiered', 'core', 'curriculum', 'help', 'equip', 'competencies', 'needed', 'work', 'within', 'public', 'health', 'emergency', 'response', 'start', 'learning', 'today', 'amp', 'hashtag', 'ready4response']

*****Stemming->Final Result*****
['intern', 'day', 'disast', 'risk', 'reduct', 'hashtag', 'openwho', 'launch', 'multi', 'tier', 'core', 'curriculum', 'help', 'equip', 'compet', 'need', 'work', 'within', 'public', 'health', 'emerg', 'respons', 'start', 'learn', 'today', 'amp', 'hashtag', 'ready4respons']


*****Original Tweet*****
#COVID19 has shown how health emergencies and disasters affect entire communities – especially those with weak health systems, and vulnerable populations like migrants, indigenous peoples, and those living in fragile humanitarian conditions. https://t.co/jpUQpnu0V1

*****Without Emojis and Links*****
#COVID19 has shown how health

KeyboardInterrupt: ignored